# Visualización de datos geoespaciales con Cartopy 🌍

Este notebook introduce el uso de **Cartopy** para visualizar datos espaciales. 
Si encuentran ejemplos antiguos en Basemap, la lógica es bastante parecida.

Veremos:
- Cómo visualizar datos geoespaciales a través de mapas con distintas proyecciones.
- Elementos y herramientas del gráfico (línea de costa, barras de colores y rangos, cuadrícula en los ejes).
- Diferentes formas de visualizar nuestros datos (`pcolormesh`, `contourf` y `contour`).

El notebook está pensado como una herramienta práctica para aprender y luego consultar cuando lo necesiten.

## Diferencia entre **Matplotlib**, **Basemap** y **Cartopy**

- **Matplotlib**: grafica en coordenadas cartesianas (x, y).  
- **Basemap**: fue la primera extensión de matplotlib para mapas, hoy está en desuso.  
- **Cartopy**: la librería actual recomendada; integra directamente con matplotlib y soporta múltiples proyecciones.  

Cartopy es más flexible y tiene soporte activo.

## 1. Instalación

Para instalar las librerías necesarias, escribimos en la terminal dentro de nuestro entorno de trabajo:

```bash
pip install xarray matplotlib cartopy cmocean
```

## 2. Importación de librerías necesarias

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cmocean
import numpy as np

## 3. Abrir nuestro archivo de datos con formato NetCDF

Usaremos un archivo NetCDF que contiene información global de temperatura superficial del mar (SST), para un día en particular, que está disponible y pueden descargar en https://data.remss.com/SST/daily/mw_ir/v05.1/netcdf/.

En este ejemplo, el archivo se llama `20251004120000-REMSS-L4_GHRSST-SSTfnd-MW_IR_OI-GLOB-v02.0-fv05.1.nc`, y tiene coordenadas `lat`,`lon` y `time`. 

In [ ]:
# Abrir dataset NetCDF
path = '/home/jovyan/shared/datos_para_tutoriales/Talleres_intermedios/7-Octubre-2025/datos_espaciales_python/20251004120000-REMSS-L4_GHRSST-SSTfnd-MW_IR_OI-GLOB-v02.0-fv05.1.nc'
ds = xr.open_dataset(path)

In [ ]:
# Seleccionamos la variable que contiene los datos de temperatura, en este caso: `analysed_sst`
sst_data = ds.analysed_sst

In [ ]:
# Seleccionamos el primer tiempo del dataset para obtener una variable espacial 2D
sst_2D = sst_data.isel(time=0)
sst_2D = sst_2D

# Pasamos a grados celsius
sst = sst_2D - 273

## 4. Visualización básica con Matplotlib (sin proyección)

Usaremos `pcolormesh` para una visualización rápida de los datos.

In [ ]:
# Definimos variables de latitud y longitud
lon = sst.lon.values
lat = sst.lat.values

plt.pcolormesh(lon, lat, sst)

In [ ]:
# Lo pongo en contexto de una figura, con título, ejes y barra de colores. Además, cambiamos el colormap.
plt.figure(figsize=(8,5))
plt.pcolormesh(lon, lat, sst, cmap=cmocean.cm.thermal) 
plt.colorbar(label="SST (°C)")
plt.title("Visualización básica con matplotlib (sin proyección)")
plt.xlabel("Longitud")
plt.ylabel("Latitud")
plt.show()

## 5. Visualización básica con Cartopy

Graficaremos el campo de temperatura en una proyección **PlateCarree** (la más simple) y agregaremos las líneas de costa.

Nota: `transform=ccrs.PlateCarree()` indica que las coordenadas de los datos están en lat/lon. Esto es importante para que Cartopy proyecte correctamente los datos sobre la proyección del mapa.

In [ ]:
fig = plt.figure(figsize=(10,6))
ax = plt.axes(projection=ccrs.PlateCarree()) 
sst.plot.pcolormesh(ax=ax, transform=ccrs.PlateCarree(), cmap=cmocean.cm.thermal,
                    cbar_kwargs={'label': 'SST (°C)'})
ax.coastlines()
ax.set_title("Temperatura superficial del mar - Proyección PlateCarree")
plt.show()

## Ejemplo con otra proyección y agregado de herramientas Cartopy

La **proyección Miller** es similar a Mercator, pero corrige ligeramente la distorsión en latitudes altas. Útil para mapas globales.

Además, definimos los límites mínimo y máximo del plot para una mejor visualización.

Y agregaremos más herramientas de cartopy: con `add_feature()` y `ax.set_extent()`

In [ ]:
fig = plt.figure(figsize=(10,6))
ax = plt.axes(projection=ccrs.Mercator()) # Defino la proyección del mapa
sst.plot.pcolormesh(ax=ax, transform=ccrs.PlateCarree(), cmap=cmocean.cm.thermal,   # Uso la transformación PlateCaree porque tengo los datos en lat/lon
                    vmin = sst.min(), vmax = sst.max(), # Valores mínimo y máximo de la sst
                    cbar_kwargs={'label': 'SST (°C)'})
ax.coastlines()
ax.gridlines(draw_labels=True)
ax.add_feature(cfeature.LAND, color = 'grey')
ax.set_title("Temperatura superficial del mar - Proyección Miller")
plt.show()

In [ ]:
lon_min = -82
lon_max = -50
lat_min = -60
lat_max = -20

fig = plt.figure(figsize=(10,6))
ax = plt.axes(projection=ccrs.Miller()) # Defino la proyección del mapa
ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())
sst.plot.pcolormesh(ax=ax, transform=ccrs.PlateCarree(), cmap=cmocean.cm.thermal,   # Uso la transformación PlateCaree porque tengo los datos en lat/lon
                    vmin = sst.min(), vmax = sst.max(), # Valores mínimo y máximo de la sst
                    cbar_kwargs={'label': 'SST (°C)'})
ax.coastlines(color = 'red')
ax.add_feature(cfeature.LAND, color = 'black')

# Gridlines
gl = ax.gridlines(draw_labels=True, linewidth=0.5, color='grey', linestyle='-.')
gl.top_labels = False 
gl.right_labels = False


ax.set_title("Temperatura superficial del mar - Proyección Miller")
plt.show()

## 6. Diferencias entre `pcolormesh`, `contour` y `contourf`

- **pcolormesh**: colorea cada celda con el valor correspondiente (ideal para datos de grilla).
- **contour**: dibuja sólo las líneas de contorno (isolíneas), sin color de relleno.
- **contourf**: crea polígonos coloreados interpolando entre valores (útil para visualizar gradientes suaves).

In [ ]:
fig, axes = plt.subplots(1,3, figsize=(18,5), subplot_kw={'projection': ccrs.PlateCarree()})

# pcolormesh
sst.plot.pcolormesh(ax=axes[0], transform=ccrs.PlateCarree(), cmap=cmocean.cm.thermal,
                    cbar_kwargs={'label': 'SST (°C)'})
axes[0].set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())
axes[0].coastlines()
axes[0].set_title("pcolormesh")

# contourf
sst.plot.contourf(ax=axes[1], transform=ccrs.PlateCarree(), cmap=cmocean.cm.thermal, vmin = sst.min(), vmax = sst.max(),
                  cbar_kwargs={'label': 'SST (°C)'})
axes[1].set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())
axes[1].coastlines()
axes[1].set_title("contourf")

# contour
sst.plot.contour(ax=axes[2], transform=ccrs.PlateCarree(), vmin = sst.min(), vmax = sst.max(), colors='black')
axes[2].set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())
axes[2].coastlines()
axes[2].set_title("contour")

plt.show()

## Figura Final Mejorada

Ejemplo de mapa con estética más cuidada y uso de herramientas de Cartopy.

In [ ]:
fig = plt.figure(figsize=(10,6))
ax = plt.subplot(111, projection=ccrs.Miller())
ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())

# Título
ax.set_title("Temperatura superficial del mar\n 2025-10-04")

# Definir niveles para contourf
levels_contourf = np.linspace(0, 20, 21)  # 21 niveles y van de 0ªC a 20 ªC

# Contourf para colores
cf = ax.contourf(lon, lat, sst, levels=levels_contourf, transform=ccrs.PlateCarree(),
                 cmap=cmocean.cm.thermal, extend='both')

# Contour para isotermas específicas
iso = ax.contour(lon, lat, sst, levels=[5, 10], colors='lightgray', linewidths=1,
                 transform=ccrs.PlateCarree())
ax.clabel(iso, fmt='%d°C', inline=True, fontsize=10)  # etiquetas en las isotermas

# Tierra con cfeature.LAND
ax.add_feature(cfeature.LAND, facecolor='lightgray', edgecolor='grey')

# Gridlines
gl = ax.gridlines(draw_labels=True, linewidth=0.5, color='grey', linestyle='-.')
gl.top_labels = False
gl.right_labels = False

# Barra de colores
cb = plt.colorbar(cf, extend="both", orientation='vertical')
cb.set_label("SST (°C)")

plt.show()

## Ejercicios

- Cambiar la zona de interés usando `ax.set_extent()`
- Cambiar la proyección de la figura
- Añadir fronteras y ríos usando `ax.add_feature`()